Use the following Azure Databricks storage setup block only if you are using Azure Databricks. You can refer to the instructions here to get started:
https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/adls-gen2/azure-datalake-gen2-sp-access

If you are using Synapse Spark and if your data is residing on the storage attached to the Synapse Spark workspace, you can skip the below storage setup section.

In [ ]:
%scala
val storageAccountName = "<INSERT STORAGE ACCOUNT>"
val fileSystemName = "<INSERT CONTAINER NAME>"

val commonPath = "abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net"

# AAD Application Details
val appID = "<INSERT APP ID>"
val secret = "<INSERT SECRET>"
val tenantID = "<INSERT TENANT ID>"

spark.conf.set("fs.azure.account.auth.type." + storageAccountName + ".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type." + storageAccountName + ".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id." + storageAccountName + ".dfs.core.windows.net", "" + appID + "")
spark.conf.set("fs.azure.account.oauth2.client.secret." + storageAccountName + ".dfs.core.windows.net", "" + secret + "")
spark.conf.set("fs.azure.account.oauth2.client.endpoint." + storageAccountName + ".dfs.core.windows.net", "https://login.microsoftonline.com/" + tenantID + "/oauth2/token")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

In [ ]:
%python
# Let us create a sample Dataframe with sensitive data
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
cols = StructType([ \
    StructField("Name",StringType(),True), \
    StructField("SSN",StringType(),True), \
    StructField("email",StringType(),True)
  ])
 
data = [("Adam Smith","111-11-1111","adam@adam.com"),
    ("Brenda Harman","222-22-2222","brenda@brenda.com"),
    ("Carmen Pinto","333-33-3333", "carmen@carmen.com")
  ]

# Create the Dataframe
piidf = spark.createDataFrame(data=data,schema=cols)
display(piidf)

To use the Fernet library, you will have to install encryption and crytography libraries. Go to the Compute tab of the Azure Databricks workspace. In the compute tab,  under Libraries  select PyPI and enter "encryption" in the textbox and click on "Install new" button. 
Repeat the same process for the "cryptography" library also.

In [ ]:
# Import the Fernet library and test it
from cryptography.fernet import Fernet
key = Fernet.generate_key()
f = Fernet(key)
token = f.encrypt(b"Hello Azure")
print(token)
print(f.decrypt(token))

In [ ]:
# Define the encryption function
def encryptUdf(plaintext, KEY):
    from cryptography.fernet import Fernet
    f = Fernet(KEY)
    encryptedtext = f.encrypt(bytes(plaintext, 'utf-8'))
    return str(encryptedtext.decode('ascii'))
encrypt = udf(encryptUdf, StringType())


# decrypt udf
def decryptUdf(encryptedtext, KEY):
    from cryptography.fernet import Fernet
    f = Fernet(KEY)
    plaintext=f.decrypt( encryptedtext.encode()).decode()
    return plaintext
decrypt = udf(decryptUdf, StringType())



In [ ]:
from pyspark.sql.functions import udf, lit, md5
from pyspark.sql.types import StringType

# Fetch key from secrets or Azure Key vault
# encryptionKey = dbutils.preview.secret.get(scope = "encrypt", key = "fernetkey")
encryptionKey = key
# Encrypt the data 
encrypteddf = piidf.withColumn("SSN", encrypt("SSN", lit(encryptionKey)))
display(encrypteddf)

#Save encrypted data 
encrypteddf.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("PIIEncryptedTable")

In [ ]:
decrypted = encrypteddf.withColumn("SSN", decrypt("SSN",lit(encryptionKey)))
display(decrypted)